In [1]:
import pandas as pd
import numpy as np

## Model Training

In [2]:
df = pd.read_csv('eda_cleaned.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Distance,Time_taken_min
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,2022-12-02 21:55:00,...,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,10.29,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,2022-02-13 14:55:00,...,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,6.25,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,2022-04-03 17:30:00,...,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,13.80,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,2022-02-13 09:20:00,...,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,2.93,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,2022-02-14 19:50:00,...,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,19.42,41


In [3]:
df=df.drop(labels=['ID','Delivery_person_ID','Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude','Order_Date','Time_Orderd','Time_Order_picked'],axis=1)

In [4]:
## Independent and dependent features
X = df.drop(labels=['Time_taken_min'],axis=1)
Y = df[['Time_taken_min']]

In [5]:
Y

,Time_taken_min
0,46
1,23
2,21
3,20
4,41
...,...
33778,32
33779,36
33780,16
33781,26


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33783 entries, 0 to 33782
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Delivery_person_Age      33783 non-null  float64
 1   Delivery_person_Ratings  33783 non-null  float64
 2   Prep_time                33783 non-null  float64
 3   Weather_conditions       33783 non-null  object 
 4   Road_traffic_density     33783 non-null  object 
 5   Vehicle_condition        33783 non-null  int64  
 6   Type_of_order            33783 non-null  object 
 7   Type_of_vehicle          33783 non-null  object 
 8   multiple_deliveries      33783 non-null  float64
 9   Festival                 33783 non-null  object 
 10  City                     33783 non-null  object 
 11  Distance                 33783 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 3.1+ MB


In [7]:
# Define which columns should be ordinal-encoded and which should be scaled
Ordinal_cols = ['Road_traffic_density']
Nominal_cols = ['Weather_conditions','Type_of_order','Type_of_vehicle','Festival','City']
numerical_cols = X.select_dtypes(exclude='object').columns

In [8]:
df.multiple_deliveries.unique()

array([3., 1., 0., 2.])

In [9]:
# Define the custom ranking for each ordinal variable
Traffic_categories = ['Low','Medium','High','Jam']
#Delivery_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
#clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [10]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.preprocessing import OneHotEncoder #Onehot encoder
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [14]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    #('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
ord_pipeline=Pipeline(
    steps=[
    #('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[Traffic_categories])),
    ('scaler',StandardScaler())
    ]

)

# Categorigal Pipeline
nom_pipeline=Pipeline(
    steps=[
    #('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehotencoder',OneHotEncoder(sparse=False)),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('ord_pipeline',ord_pipeline,Ordinal_cols),
('nom_pipeline',nom_pipeline,Nominal_cols),
])


In [15]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [ ]:
cols_op = pd.Series(list(numerical_cols) + list(categorical_cols))
cols_op

In [16]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

AttributeError: Estimator ordinalencoder does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?

In [ ]:
X_train.head()

In [ ]:
X_train.head()

In [ ]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
regression=LinearRegression()
regression.fit(X_train,y_train)

In [ ]:
regression.coef_

In [ ]:
regression.intercept_

In [ ]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


In [ ]:
model_list